<h2>Multi-Blockchain Wallet in Python</h2>
<h3>Importing Libraries</h3>  

In [38]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account

<h3>Web3 Connections</h3>

In [2]:
#Nodes runing with POW
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))

# Loading EV
load_dotenv("Mnemonic.env")

# Loading Mnemonic EV and (set this mnemonic as an environment variable, and include the one you generated as a fallback using)
mnemonic = os.getenv("Mnemonic")
print(mnemonic)

bleak decrease opera same embrace hair fitness misery neck depart process budget


<h3>Create Functions</h3>

In [3]:
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys

In [4]:
#Setting dictionary of coins to be used in the wallet

coins = {"eth", "btc-test", "btc"}
numderive = 3

In [5]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [6]:

eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0x47a823eff019e920d4468ad6f250371fed8bc8639703938dbad1b8795a8c1b08"
"cV2qTnb8FkWmsvBoVRqkSaX262Mcuawop2T9ttvwZVTK4w46wdhw"


In [7]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "1Kp5zkgRgv2enRjnNT966B2c3sPWhatcxi",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KyQHh7gKE5XqHHJoZQfgnT75GNUBwdpDzdPfwKQGTAQRJJ9LdcoU",
            "pubkey": "03982d837514d9b1766bf1734e0cc345e60b606a9677cec844a65a8702ad3e8989",
            "pubkeyhash": "ce59fc0b5d5fb7b6ef8e96d67fc08e3774250588",
            "xprv": "xprvA2yEnQRxChBR1v7B8Hc9muD7sMT6Vxc4vFKfavNxTWtjWw4dXuZwJe4n64bH5HFRLEer1VRNQLVnjH4CPGee9SNzMTJvDRm35YPhFd4sfie",
            "xpub": "xpub6FxbBuxr34jiEQBeEK9A939rRPHauRKvHUFGPJna1rRiPjPn5StBrSPFwNE8ZS9efMqiqYwqwVJZL64U9c3b6g2WHyAEPnx2XTvKmVxgVBJ"
        },
        {
            "address": "1KvPqCzqzR3JkfuxT14UCWuAdAU41rDpTP",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L4C8GeapV9EDJMy6Can4y7FHw1CqouuycxKwsotkPj4ywKpaxx7s",
            "pubkey": "033a824205eb595e7568321eaadb7875e42134d956f18017a41c49576b6a97a1d8",
            "pubkeyhash":

In [8]:
def priv_key_to_account(coin, priv_key):
    """Convert the privkey string in a child key to an account object that bit or web3.py can use to transact"""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

In [9]:
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: mqs3swRWNyASkUSuutLu9SqyqFqJMjUAcr>


In [23]:
def create_trx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global trx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [24]:
def send_trx(coin, account, recipient, amount):
    """call create_trx, sign the transaction, then send it to the designated network"""
    if coin == "eth": 
        trx_eth = create_trx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_trx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

<h3>Create Transactions</h3>
<h3>BTC Transaction</h3>

In [32]:
# create BTC transaction
create_trx(BTCTEST,btc_acc,"mqs3swRWNyASkUSuutLu9SqyqFqJMjUAcr", 0.00001)

'{"unspents":[{"amount":80000,"confirmations":0,"script":"76a914717e3d11a9ebea2d5423d8b520542f16962ca7ff88ac","txid":"54c1bef046429ab830c4ed463e1992675f92da4f4b40fd4c3a1dc7518a4a6e65","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mqs3swRWNyASkUSuutLu9SqyqFqJMjUAcr",1000],["mqs3swRWNyASkUSuutLu9SqyqFqJMjUAcr",55948]]}'

In [34]:
#send BTC transaction
send_trx(BTCTEST,btc_acc,'mqs3swRWNyASkUSuutLu9SqyqFqJMjUAcr',0.0001)

'0100000001656e4a8a51c71d3a4cfd404b4fda925f6792193e46edc430b89a4246f0bec154000000006b4830450221009ff192c697fade764e5af78a79476c36d82f9d8faac4324637edf9c2f94915f602205f183a684fa460acaedd9c09763d66305206dd2f29befa887ac4e831754ffe7901210227584db60a55f124818abccc59196d42e413370d4211fb08f7e88faa6b9c6d45ffffffff0210270000000000001976a914717e3d11a9ebea2d5423d8b520542f16962ca7ff88ac64b70000000000001976a914717e3d11a9ebea2d5423d8b520542f16962ca7ff88ac00000000'

<h3>ETH Transaction</h3>

In [25]:
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
from web3 import Web3, HTTPProvider
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x47a823eff019e920d4468ad6f250371fed8bc8639703938dbad1b8795a8c1b08"))
w3.isConnected()

True

In [40]:
w3.eth.getBalance("0x9d0Ae1330EbD84Ac9a2055A1AD6C92C459b6a846")

3999159999999999989

In [41]:
create_trx(ETH,eth_acc,"0x35031eDcF3e4932f9B8AA6067cd8E562F5c863Cf", 100)

{'to': '0x35031eDcF3e4932f9B8AA6067cd8E562F5c863Cf',
 'from': '0x9d0Ae1330EbD84Ac9a2055A1AD6C92C459b6a846',
 'value': 100,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 2}

In [42]:
send_trx(ETH, eth_acc,"0x35031eDcF3e4932f9B8AA6067cd8E562F5c863Cf", 100)

0x7687ec7ffafa04aed021b089abf2dfd0a9344997d70301aaa9fecf12d0c78bf1


'0x7687ec7ffafa04aed021b089abf2dfd0a9344997d70301aaa9fecf12d0c78bf1'

<h3>Verification of ETH Transaction</h3>

In [43]:
w3.eth.getBalance("0x9d0Ae1330EbD84Ac9a2055A1AD6C92C459b6a846")

3998739999999999889

<h3>Nice!</h3>